In [ ]:
# import our libraries
import re
import requests
import unicodedata
from bs4 import BeautifulSoup
import sys
from pathlib import Path
sys.setrecursionlimit(10000)
from tqdm import tqdm
import json
from bs2json import bs2json
import pickle


def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)


for file in tqdm(Path('data/data').glob("*.txt")):
    try:
        with open(file,'r') as f:
            soup = BeautifulSoup(f.read(), 'lxml')
        done = False
        for filing_document in soup.find_all('document'):
            document_id = filing_document.type.find(text=True, recursive=False).strip()
            if document_id == "10-K":
                document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
                document_filename = filing_document.filename.find(text=True, recursive=False).strip()
                try:
                    document_description = filing_document.description.find(text=True, recursive=False).strip()
                except:
                    document_description = ""
    
                filing_doc_text = filing_document.find('text').extract()
                all_thematic_breaks = filing_doc_text.find_all('hr',{'width':'100%'})
                all_page_numbers = []
                all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]
                filing_doc_string = str(filing_doc_text)
                if len(all_thematic_breaks) > 0:
                    regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))
                    split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)
                    pages_code = split_filing_string
                
                elif len(all_thematic_breaks) == 0:
                    split_filing_string = all_thematic_breaks
                    pages_code = [filing_doc_string]

                document_pages = pages_code
                pages_length = len(pages_code)
                repaired_pages = {}
                normalized_text = {}
                for index, page in enumerate(document_pages):
                    page_soup = BeautifulSoup(page,'lxml')
                    try:
                        page_text = page_soup.html.body.get_text(' ',strip = True)
                    except : 
                        page_text = ""
                    try:
                        page_text_norm = restore_windows_1252_characters(unicodedata.normalize('NFKD', page_text)) 
                    except:
                        page_text_norm = ""
                    try:
                        page_text_norm = page_text_norm.replace('  ', ' ').replace('\n',' ')
                    except:
                        page_text_norm = ""

                    page_number = index + 1
                    normalized_text[page_number] = page_text_norm
                    repaired_pages[page_number] = page_soup

                pages_normalized_text = normalized_text

                # add the repaired html code back to the document dictionary
                pages_code = repaired_pages
                name = str(file).split('/')[-1].split('.')[0]
                with open(f'data/htmls/{name}.html','w') as f:
                    f.write(str(pages_code[1]))
                with open(f'data/normalised/{name}.txt','w') as f:
                    f.write(pages_normalized_text[1])
                done = True
                break
        if done == False:
            with open('not_done.txt','a') as f:
                f.write(file+" 10-k not found\n")

    except:
        with open('not_done.txt','a') as f:
            f.write(str(file)+"\n")






In [ ]:
with open("data/htmls/175.html",'r') as f:
    soup = BeautifulSoup(f.read(), 'lxml')

In [ ]:
from bs4 import BeautifulSoup
from bs2json import bs2json
from tqdm import tqdm
from pathlib import Path

already_done = [str(i).split('/')[-1].split('.')[0] for i in Path('data/text_only').glob("*.txt")]
converter = bs2json()
final = []
def f(i):
    global final
    if type(i) == dict:
        for j in i:
            if type(i[j]) == dict or type(i[j]) == list:
                f(i[j])
            elif j == "text" and i[j] != "":
                final.append(i[j])
                
    elif type(i) == list:
        for j in i:
            if type(j) == dict or type(j) == list:
                f(j)

for i in tqdm(Path('data/htmls').glob("*.html")):
    name = str(i).split('/')[-1].split('.')[0]
    if name not in already_done:
        with open(i,'r') as f1:
            soup = BeautifulSoup(f1.read(), 'lxml')


        js = converter.convert(soup.find(''))
        final = []
        f(js)
        with open(f"data/text_only/{name}.txt",'w') as f2:
            for j in final:
                f2.write(j + "\n")
    else:
        print('working')